In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

In [25]:
# create a density matrix of size 2 usinf Fock base preparation

RHO0 = fock_dm(2)
print(RHO0)

# creation of ket in ground state
phi0 = basis(3,0)
print(phi0)

# diagonal operator
print(qeye(3))

# a operator in the Fock basis of dimention N
print(destroy(2))

# Number operator in the Fock basis of dimension N
print(num(2))

# build collapse operator which is a "list"
c_ops = []
c_ops.append(0)
c_ops.append(0.5)
c_ops.append(1)
print(type(c_ops))
print(c_ops)


#print(phi0*phi0.dag())
#help(tensor)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]]
Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0. 1.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]
<class 'list'>
[0, 0.5, 1]


In [12]:
# building a Hamiltoninan using 
H = 2*np.pi * 0.1 * sigmax()

In [ ]:

# definition os the ket of dimension 2 : basis(2, 0)
psi0 = basis(2, 0)

times = np.linspace(0.0, 10.0, 100)

result = mesolve(H, psi0, times, [np.sqrt(0.05) * sigmax()], [sigmaz(), sigmay()])

fig, ax = plt.subplots()

ax.plot(times, result.expect[0]) 

ax.plot(times, result.expect[1]) 

ax.set_xlabel('Time') 

ax.set_ylabel('Expectation values') 

ax.legend(("Sigma-Z", "Sigma-Y"))  

plt.show() 

In [29]:
# definition of a time vaying coefficient for Time dependant Hamiltoninan solver
def H1_coeff(t,arg):
    return 9 * np.exp( -(t/5.) ** 2 )

In [58]:
# exemple of Time varying Hamiltoninan
ground  = basis(3, 0)
ustate  = basis(3, 1)
excited = basis(3, 2)

N = 2 # Set where to truncate Fock state for cavity
# build the tensorial operator space
sigma_ge = tensor(qeye(N), ground * excited.dag())  # I x |g><e|
sigma_ue = tensor(qeye(N), ustate * excited.dag())  # I x |u><e|
a        = tensor(destroy(N), qeye(3))              # a x I
ada      = tensor(num(N), qeye(3))                  # ada x I

c_ops = []  # Build collapse operators as a list of successive loss terms
kappa = 1.5 # Cavity decay rate
gamma = 6  # Atomic decay rate
c_ops.append(np.sqrt(kappa) * a) # destruction operator : losses in cavity which do no affect atoms
c_ops.append(np.sqrt(4*gamma/9) * sigma_ge) # atomic decays from e to g
c_ops.append(np.sqrt(5*gamma/9) * sigma_ue) # atomic decays from e to u

t = np.linspace(-15, 15, 100) # Define time vector
psi0 = tensor(basis(N, 0), ustate) # Define initial state

g = 5  # coupling strength
H0 = -g * (sigma_ge.dag() * a + a.dag() * sigma_ge)  # time-independent term
H1 = (sigma_ue.dag() + sigma_ue)                     # time-dependent term

H = [H0,[H1, H1_coeff]] # define the time-varying Hamiltonian


# define the projection dictorary
state_GG = tensor(basis(N, 1), ground) # Define states onto which to project
sigma_GG = state_GG * state_GG.dag()
state_UU = tensor(basis(N, 0), ustate)
sigma_UU = state_UU * state_UU.dag()

# [ada, sigma_UU, sigma_GG] : Dictionary of scope parameters for time-dependent Hamiltonians
#output = mesolve(H, psi0, t, c_ops, [ada, sigma_UU, sigma_GG])

opts = Options()
opts.store_states = True
opts.store_final_state = True
print(opts)

output = mesolve(H, psi0, t, c_ops, options=opts)

# attribute of output (check : https://qutip.org/docs/latest/apidoc/classes.html#qutip.solver.Options )
# times : Times at which simulation data was collected
# expect: list/array Expectation values (if requested) for simulation.
# states : array , State of the simulation (density matrix or ket) evaluated at times.

print(output)

# fig, ax = plt.subplots()

# ax.plot(t, output.expect[0]) 

# ax.plot(t, output.expect[1]) 

# ax.set_xlabel('Time') 

Options:
-----------
atol:              1e-08
rtol:              1e-06
method:            adams
order:             12
nsteps:            1000
first_step:        0
min_step:          0
max_step:          0
tidy:              True
num_cpus:          8
norm_tol:          0.001
norm_steps:        5
rhs_filename:      None
rhs_reuse:         False
seeds:             0
rhs_with_state:    False
average_expect:    True
average_states:    False
ntraj:             500
store_states:      True
store_final_state: True

[-15.         -14.6969697  -14.39393939 -14.09090909 -13.78787879
 -13.48484848 -13.18181818 -12.87878788 -12.57575758 -12.27272727
 -11.96969697 -11.66666667 -11.36363636 -11.06060606 -10.75757576
 -10.45454545 -10.15151515  -9.84848485  -9.54545455  -9.24242424
  -8.93939394  -8.63636364  -8.33333333  -8.03030303  -7.72727273
  -7.42424242  -7.12121212  -6.81818182  -6.51515152  -6.21212121
  -5.90909091  -5.60606061  -5.3030303   -5.          -4.6969697
  -4.39393939  -4.09090909 